![openclassrooms](https://s3.eu-west-1.amazonaws.com/course.oc-static.com/courses/6204541/1+HnqdJ-5ofxiPP9HIxdNdpw.jpeg)

# Agrégez des données avec Pandas

Suite aux résultats que vous avez fourni précédemment, le responsable du service prêt revient vous voir avec une remarque : vous l’avez probablement remarqué, mais certains clients ont contracté plusieurs prêts au sein de notre établissement. Cela fausse donc potentiellement les calculs réalisés précédemment.

Il formule donc des demandes additionnelles, auxquelles vous allez devoir répondre en appliquant diverses agrégations.

In [2]:
import numpy as np
import pandas as pd

In [3]:
# traitement réalisés précédemment
prets = pd.read_csv('https://raw.githubusercontent.com/OpenClassrooms-Student-Center/fr-4452741-decouvrez-les-librairies-python-pour-la-data-science/main/data/prets.csv')

# calcul du taux d'endettement
prets['taux_endettement'] = round(prets['remboursement'] * 100 / prets['revenu'], 2)

# renommer taux en taux_interet
prets.rename(columns={'taux':'taux_interet'}, inplace=True)

# calculer le cout total du pret
prets['cout_total'] = prets['remboursement'] * prets['duree']

# calculer les bénéfices mensuels réalisés
prets['benefices'] = round((prets['cout_total'] * prets['taux_interet']/100)/(24), 2)

# création d'une variable risque
prets['risque'] = 'Non'
prets.loc[prets['taux_endettement'] > 35, 'risque'] = 'Oui'

prets.head()

,identifiant,ville,CP,revenu,remboursement,duree,type,taux_interet,taux_endettement,cout_total,benefices,risque
0,0,TOULOUSE,31100,3669.0,1130.05,240,immobilier,1.168,30.80,271212.0,131.99,Non
1,1,PARIS,75009,5310.0,240.00,64,automobile,3.701,4.52,15360.0,23.69,Non
2,1,PARIS,75009,5310.0,1247.85,300,immobilier,1.173,23.50,374355.0,182.97,Non
3,2,MARSEILLE,13010,1873.0,552.54,240,immobilier,0.972,29.50,132609.6,53.71,Non
4,3,MARSEILLE,13010,1684.0,586.03,180,immobilier,1.014,34.80,105485.4,44.57,Non


Pourriez vous créez un dataframe profil client pour pallier ce problème ?

Par profil client, nous entendons un dataframe où il y n'y a qu'une seule ligne par client, avec le résumé de ses informations (somme remboursement, du taux d'endettement, du cout_total et des bénéfices réalisés)

In [5]:
profil_client = prets.groupby('identifiant').agg({
    'remboursement': 'sum',
    'taux_endettement': 'sum',
    'cout_total': 'sum',
    'benefices': 'sum'
})
profil_client.head()

,remboursement,taux_endettement,cout_total,benefices
identifiant,,,,
0,1130.05,30.80,271212.0,131.99
1,1487.85,28.02,389715.0,206.66
2,552.54,29.50,132609.6,53.71
3,586.03,34.80,105485.4,44.57
4,423.61,28.70,101666.4,51.21


Recalculez le nombre exact de personnes en situation bancaire risquée à partir du taux d'endettemment :

In [7]:
clients_risque = profil_client[profil_client['taux_endettement'] > 35]
nombre_clients_risque = clients_risque.shape[0]
print("Nombre de clients à risque :", nombre_clients_risque)

Nombre de clients à risque : 62


Calculez le bénéfice dégagé par chacune de nos agences, par types de prêts. Vous présenterez vos résultats sous la forme d'un tableau simple (via un `group_by`) :

In [8]:
prets.groupby(['ville', 'type'])['benefices'].sum()

ville      type      
BORDEAUX   immobilier    1681.32
LYON       automobile      47.39
           immobilier    2967.41
MARSEILLE  automobile      61.36
           immobilier    4334.55
NICE       automobile      31.15
           immobilier    2651.60
PARIS      automobile      99.50
           immobilier    9916.85
TOULOUSE   automobile      61.22
           immobilier    2076.85
Name: benefices, dtype: float64

Pour aller plus loin, on souhaite avoir un tableau à double entrée (via un `pivot_table`) présentant cette fois ci les bénéfices moyen réalisés par chaque agence, pour chaque type de prêt :

In [9]:
pivot_table = pd.pivot_table(prets,
                             values='benefices',
                             index='ville',        # Chaque ligne correspond à une agence
                             columns='type',       # Chaque colonne correspond à un type de prêt
                             aggfunc='mean')       # Calcul de la moyenne des bénéfices
print(pivot_table)

type       automobile  immobilier
ville                            
BORDEAUX          NaN  105.082500
LYON        15.796667  118.696400
MARSEILLE   20.453333   60.202083
NICE        15.575000  139.557895
PARIS       19.900000  137.734028
TOULOUSE    20.406667   86.535417


Quelle ville semble la plus intéressante où développer les prêts immobiliers ?

Félicitation pour être arrivé au bout ! Voici [la correction correspondante](https://colab.research.google.com/github/OpenClassrooms-Student-Center/fr-4452741-decouvrez-les-librairies-python-pour-la-data-science/blob/main/notebooks/P2/P2C4%20-%20Agr%C3%A9gez%20des%20donn%C3%A9es%20avec%20Pandas%20-%20correction.ipynb).

In [10]:
pivot_table_sorted = pivot_table.sort_values(by='immobilier', ascending=False)
print(pivot_table_sorted.head(1))


type   automobile  immobilier
ville                        
NICE       15.575  139.557895
